In [32]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset

In [2]:
dataset = load_dataset("daily_dialog")

Using custom data configuration default
Reusing dataset daily_dialog (/home/pavel/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset['train'].flatten()

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 11118
})

In [10]:
dataset['train'].to_csv('train_dataset.csv')

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

6286926

In [9]:
dir(dataset['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_split',
 'add_column',
 'add_elasticsearch_index',
 'add_faiss_index',
 'add_faiss_index_from_external_arrays',
 'add_item',
 'align_labels_with_mapping',
 'builder

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('train_dataset.csv')

In [13]:
df.head(3)

,Unnamed: 0,dialog,act,emotion
0,0,"['Say , Jim , how about going for a few beers ...",[3 4 2 2 2 3 4 1 3 4],[0 0 0 0 0 0 4 4 4 4]
1,1,"['Can you do push-ups ? '\n "" Of course I can ...",[2 1 2 2 1 1],[0 0 6 0 0 0]
2,2,"['Can you study with the radio on ? '\n ' No ,...",[2 1 2 1 1],[0 0 0 0 0]


In [16]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [20]:
df.dtypes

dialog     object
act        object
emotion    object
dtype: object

In [22]:
dataset['train'].to_dict()

{'dialog': [['Say , Jim , how about going for a few beers after dinner ? ',
   ' You know that is tempting but is really not good for our fitness . ',
   ' What do you mean ? It will help us to relax . ',
   " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
   " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
   ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
   " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
   ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
   " Good.Let ' s go now . ",
   ' All right . '],
  ['Can you do push-ups ? ',
   " Of course I can . It's a piece of cake ! Believe it or not , I can do 30 push-ups a minute . ",
   " Really ? I think that's impossible ! ",
   ' You mean 30

In [41]:
def explode_dataset(dataset):
    as_dict = dataset.to_dict()
    dialog, act, dialog_num = [], [], []
    for i, (d_arr, act_arr) in enumerate(zip(as_dict['dialog'], as_dict['act'])):
        if len(d_arr) != len(act_arr):
            raise ValueError('Size of arrays should be equal')
        dialog += d_arr
        act += act_arr
        dialog_num += [i] * len(d_arr)
    if len(dialog) != len(act):
        raise ValueError('')
    return {'dialog': dialog, 'act': act, 'dialog_num': dialog_num}

In [24]:
explode_dataset(dataset['train'])

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ',
  ' You know that is tempting but is really not good for our fitness . ',
  ' What do you mean ? It will help us to relax . ',
  " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
  " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
  ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
  " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
  ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
  " Good.Let ' s go now . ",
  ' All right . ',
  'Can you do push-ups ? ',
  " Of course I can . It's a piece of cake ! Believe it or not , I can do 30 push-ups a minute . ",
  " Really ? I think that's impossible ! ",
  ' You mean 30 push-ups ? ',


In [42]:
train = Dataset.from_dict(explode_dataset(dataset['train']))
test = Dataset.from_dict(explode_dataset(dataset['test']))
validation = Dataset.from_dict(explode_dataset(dataset['validation']))

In [43]:
exploded_dataset = DatasetDict(train=train, test=test, validation=validation)

In [44]:
exploded_dataset

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'dialog_num'],
        num_rows: 87170
    })
    test: Dataset({
        features: ['dialog', 'act', 'dialog_num'],
        num_rows: 7740
    })
    validation: Dataset({
        features: ['dialog', 'act', 'dialog_num'],
        num_rows: 8069
    })
})

In [47]:
train[40]

{'dialog': " That ’ s bogus . I don't feel any stress at work , and my love life is practically nonexistent . This zodiac stuff is all a bunch of nonsense . ",
 'act': 1,
 'dialog_num': 5}

In [49]:
exploded_dataset.save_to_disk('exploded_dataset')

In [50]:
e=DatasetDict.load_from_disk('exploded_dataset')

In [52]:
import datasets

In [53]:
datasets.list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'f1',
 'gleu',
 'glue',
 'google_bleu',
 'indic_glue',
 'matthews_correlation',
 'meteor',
 'pearsonr',
 'precision',
 'recall',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'wer',
 'wiki_split',
 'xnli']